In [37]:
import bql
import numpy as np
from IPython.display import display, clear_output
import pandas as pd
import time 
import datetime
from ipywidgets import *
from bqplot import *
from bqplot.interacts import *

from matplotlib import pyplot as plt
from scipy.cluster.hierarchy import dendrogram, linkage

bq = bql.Service()
d = bq.data
f = bq.func
u = bq.univ

In [38]:
# INPUTS ###############

index_universe = ['HFRIFOF Index', 'HFRIEHI Index', 'HFRIFWI Index', 'HFRIMI Index', 'HFRIFOFD Index', 'HFRIEMNI Index', 'HFRIDSI Index', 'HFRIEDI Index', 'HFRIFOFC Index',
                  'HFRIMAI Index', 'HFRIRVA Index', 'HFRIFI Index', 'HFRIMTI Index', 'HFRICAI Index', 'HFRIFIMB Index', 'HFRICRDT Index', 'HFRIMDT Index', 'HFRIACT Index',
                  'HFRIFIHY Index', 'HFRIVOL Index', 'HFRIAWC Index', 'HFRIMMS Index', 'HFRIFOFM Index', 'HFRIEDSS Index', 'HFRIWRLD Index', 'HFRICRED Index', 'HFRIAWEH Index',
                  'HFRIFWIE Index', 'HFRIFOFS Index', 'HFRIENHI Index', 'HFRIEM Index', 'HFRISTI Index', 'HFRIMCUR Index', 'HFRIAWED Index', 'HFRIEMG Index', 'HFRIFISV Index',
                  'HFRIEMA Index', 'HFRIEHFV Index', 'HFRIEDMS Index', 'HFRIEHMS Index', 'HFRIAWJ Index', 'HFRIMCOM Index', 'HFRIFWIC Index', 'HFRIAWM Index', 'HFRISEN Index',
                  'HFRIJPN Index', 'HFRIMACT Index', 'HFRIEHFG Index', 'HFRIFWIJ Index', 'HFRISRE Index', 'HFRIEMLA Index', 'HFRICHN Index', 'HFRIWOMN Index', 'HFRIIND Index',
                  'HFRIAWRV Index', 'HFRIFWIG Index', 'HFRIWEU Index', 'HFRICIS Index', 'HFRIDVRS Index', 'HFRINA Index']

# index_universe = ['HFRIEDI Index', 'HFRIEHI Index', 'HFRIFOF Index', 'HFRIFWI Index', 'HFRIMDT Index', 'HFRIMTI Index', 'HFRIRVA Index']

start_date = datetime.datetime(2008,12,31)
end_date = datetime.datetime(2018,12,31)

In [39]:
def norm_corr(as1, as2, st_date, e_date):
    price = d.px_last(start = st_date, end = e_date, per = 'M', fill='prev')
    price = f.zscore(price)
    base_price = f.value(price, u.list(as1))
    correlation = f.corr(base_price, price)

    request = bql.Request(as2, {'Cor': correlation}, preferences={'currencycheck':'ignore'}) #There's an error with de currencies of these tickers. This way we "solve" the issue
    res = bq.execute(request)
    cor_res = res[0].df()
    
    return cor_res.iloc[0]['Cor']

def correlDist(corr):
    dist = ((1-corr)/2)**0.5
    return dist

In [40]:
###############    MAIN    ###############

#Calculate Cluster
count = 0
for asset1 in index_universe:
    aux_cor_list=[]
    for asset2 in index_universe:
        aux_cor_list.append(norm_corr(asset1,asset2,start_date,end_date))
        
    if count == 0:
        Correl_Matrix = pd.DataFrame({asset1: pd.Series(aux_cor_list, index = index_universe)})
    else:
        aux_Correl_Matrix = pd.DataFrame({asset1: pd.Series(aux_cor_list, index = index_universe)})
        Correl_Matrix = pd.concat([Correl_Matrix, aux_Correl_Matrix],axis = 1)
        
    count += 1

Z = linkage(correlDist(Correl_Matrix))

plt.figure(figsize=(30, 10), facecolor='w')
dendrogram(Z, labels = index_universe)
plt.show()

In [42]:
plt.savefig('cluster_Universe.png')

In [43]:
import os
print(os.getcwd())

C:\Users\matlabidp\AppData\Local\bipy\21337033\projects\65226b93ef834601856a5de13c45ebfd


In [45]:
dendrogram(Z, labels = index_universe).savefig('cluster_Universe.png')

AttributeError: 'dict' object has no attribute 'savefig'